In [1]:
import pandas as pd
import numpy as np

In [23]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Embedding,Bidirectional
from sklearn import preprocessing 
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.utils import to_categorical
import os
from nltk.corpus import stopwords
import re
import warnings
warnings.filterwarnings('ignore')

In [29]:
train = pd.read_csv('jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('validation.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [30]:
train.shape

(223549, 8)

In [31]:
train.drop(['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.head()


,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [32]:
train['toxic'].value_counts()

0    202165
1     21384
Name: toxic, dtype: int64

In [33]:
train = train[:20000]

In [34]:
train.shape

(20000, 2)

In [35]:
train['toxic'].value_counts()

0    18053
1     1947
Name: toxic, dtype: int64

In [36]:
validation.shape

(8000, 4)

In [37]:
stop_words = set(stopwords.words('english'))

def data_text_preprocess(total_text, ind, col):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
        total_text = total_text.lower()
        
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            if not word in stop_words:
                string += word + " "
        
        train[col][ind] = string
        
for index, row in train.iterrows():
    if type(row['comment_text']) is str:
        data_text_preprocess(row['comment_text'], index, 'comment_text')

In [38]:
train.head()

,comment_text,toxic
0,explanation edits made username hardcore metal...,0
1,aww matches background colour seemingly stuck ...,0
2,hey man really trying edit war guy constantly ...,0
3,make real suggestions improvement wondered sec...,0
4,sir hero chance remember page,0


In [39]:
xtrain, xvalid, ytrain, yvalid = train_test_split( train['comment_text'].values, 
                                                   train['toxic'].values,
                                                   test_size=0.2, shuffle=True)
print(xtrain.shape)
print(xvalid.shape)
print(ytrain.shape)
print(yvalid.shape)

(16000,)
(4000,)
(16000,)
(4000,)


In [40]:
token = text.Tokenizer(num_words=None)
token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

In [41]:
max_len = 100
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

In [42]:
xtrain_pad[200]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,  2722,   745,  2669,  1501,  2826, 14879,   408,   885,
        1732, 26755, 26756,  2237,  2679, 26757,   411,   283,     1,
        7662])

In [43]:
word_index = token.word_index

#### Build The Model

In [44]:
model = Sequential()
model.add(Embedding(len(word_index)+1, 256, input_length=max_len))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
    
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(xtrain_pad, ytrain, 
          epochs=5, batch_size = 256, 
          validation_data=(xvalid_pad, yvalid))

Epoch 1/5
63/63 [==============================] - 66s 994ms/step - loss: 0.4157 - accuracy: 0.8777 - val_loss: 0.2193 - val_accuracy: 0.9118
Epoch 2/5
63/63 [==============================] - 65s 1s/step - loss: 0.1497 - accuracy: 0.9414 - val_loss: 0.1584 - val_accuracy: 0.9465
Epoch 3/5
63/63 [==============================] - 65s 1s/step - loss: 0.0570 - accuracy: 0.9841 - val_loss: 0.1809 - val_accuracy: 0.9440
Epoch 4/5
63/63 [==============================] - 76s 1s/step - loss: 0.0234 - accuracy: 0.9945 - val_loss: 0.2060 - val_accuracy: 0.9435
Epoch 5/5
63/63 [==============================] - 74s 1s/step - loss: 0.0097 - accuracy: 0.9979 - val_loss: 0.2542 - val_accuracy: 0.9430


In [46]:
scores = model.predict(xvalid_pad)

test_loss, test_acc = model.evaluate(scores,  yvalid, verbose=1)
print('\nTest accuracy:', test_acc)

125/125 [==============================] - 1s 2ms/step - loss: 0.5586 - accuracy: 0.9003

Test accuracy: 0.9002500176429749


In [47]:
y_pred = model.predict_classes(xvalid_pad)
print(len(y_pred))
print(y_pred.shape)
print(yvalid.shape)

4000
(4000, 1)
(4000,)


In [48]:
yval1 = yvalid.reshape(-1,1)
yval1.shape

(4000, 1)

In [49]:
from sklearn.metrics import confusion_matrix, classification_report

In [50]:
print(confusion_matrix(yvalid,y_pred))

[[3512   89]
 [ 139  260]]


In [51]:
print(classification_report(yvalid,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      3601
           1       0.74      0.65      0.70       399

    accuracy                           0.94      4000
   macro avg       0.85      0.81      0.83      4000
weighted avg       0.94      0.94      0.94      4000



In [ ]:
test = pd.read_csv('test.csv')
test.head()

In [ ]:
def data_text_preprocess(total_text, ind, col):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
        total_text = total_text.lower()
        
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            if not word in stop_words:
                string += word + " "
        
        test[col][ind] = string
        
for index, row in test.iterrows():
    if type(row['content']) is str:
        data_text_preprocess(row['content'], index, 'content')

In [ ]:
test_data = token.texts_to_sequences(test.content)
test_data_seq = sequence.pad_sequences(test_data, maxlen=max_len)

In [ ]:
test['toxic'] = model.predict(test_data_seq, verbose=1)

In [ ]:
test[['id', 'toxic']].to_csv('submission.csv', index=False)